# COMP30760 - Assignment 1 - Student ID: 19206730

## Task 1

This notebook covers Task 1 - Data Collection. In this first task, data is collected from the Genius open web API.

In [13]:
import json, requests, urllib
from pathlib import Path
import pandas as pd

The function below takes in an artist name as argument and fetches this artist's associated json data from the Genius API with using the previously generated unique client_access_token. The function returns the data it fetched from the API in json format to be put into a file later.

In [14]:
def fetch(search_artist):
    client_access_token = "sajg5Xa8VHfr7n7-muGj9ENce71adu8ns-vLVvKgYR_ecbVQfXz46O5vsmYCTrcA"
    genius_url = f"http://api.genius.com/search?q={search_artist}&access_token={client_access_token}"
    response = requests.get(genius_url)
    jdata = response.json()
    return jdata

Using the pathlib library, a directory is created to store different artists' song and lyrics information:

In [15]:
dir_raw = Path("genius_raw")
dir_raw.mkdir(parents=True, exist_ok=True)

A dictionary is created with some of the chosen artists in Spotify's "Top 50 Streamed Artists" records with their corresponding rankings. A for loop goes through this dictionary's keys (names of the artists) to create files that contains json data from the Genius API for each artist. Depending on the number of artists in this dictionary that has to be looped for, running this tab can take a couple seconds.

In [16]:
artist_names = {"Drake":1, "Ed Sheeran":3,  "The Weeknd":4, "Taylor Swift":5, "Justin Bieber":6, "Ariana Grande":7, 
                "Eminem":8, "Post Malone":9, "Billie Eilish":13, "XXXTENTACION":18, "Khalid":20, "Rihanna":21, 
                "Travis Scott":23, "Bruno Mars":26, "Beyonce":32, "Lady Gaga": 37, "J. Cole": 38, "Nicki Minaj": 39, 
                "Adele": 40}

for artist in artist_names.keys():
    path = "/Users/idilbilgic/Desktop/STAGE3.1/COMP30760/assignment1/genius_raw/artist" + str(artist_names[artist]) + ".json"
    with open(path, "w") as f:
        json.dump(fetch(artist), f, indent=2)

Using a double for loop, each file is looped through to find information about the particular artists' song titles, song release dates, pyong (meaning upvote) and pageview counts, and a Boolean True or False is assigned regarding if the song has other featured artists. The information is represented in a dataframe to be used for a deeper analysis in Task 2 considering trends between different artists and trends in individual artists' songs.

** note: here, the data is cleaned for a better representation, especially for the column "Release Date". **

In [17]:
metadata_rows = []

for artist in artist_names:
    artist_path = "/Users/idilbilgic/Desktop/STAGE3.1/COMP30760/assignment1/genius_raw/artist" + str(artist_names[artist]) + ".json"
    with open(artist_path, "r") as fr: 
        artist_data = json.load(fr)

    for info in artist_data["response"]["hits"]:
        row = {"Artist Name": artist,"Song Title": info["result"]["title_with_featured"]}
        row["Pyong Count"] = info["result"]["pyongs_count"]
        row["Page Views"] = info["result"]["stats"]["pageviews"]
        row["Release Date"] = str(info["result"]["release_date_components"]["year"]) + "-" + str(info["result"]["release_date_components"]["month"]) + "-" + str(info["result"]["release_date_components"]["year"])
        if info["result"]["featured_artists"] != []:
            row["Featured"] = True
        else: 
            row["Featured"] = False
        metadata_rows.append(row)

df = pd.DataFrame(metadata_rows)
df.set_index("Artist Name", inplace=True)
df

,Song Title,Pyong Count,Page Views,Release Date,Featured
Artist Name,,,,,
Drake,God's Plan,932,9236547,2018-1-2018,False
Drake,In My Feelings,212,7459807,2018-6-2018,False
Drake,Hotline Bling,644,6960256,2015-7-2015,False
Drake,One Dance (Ft. Kyla & WizKid),233,6041789,2016-4-2016,True
Drake,"Hold On, We're Going Home (Ft. Majid Jordan)",506,5507582,2013-8-2013,True
...,...,...,...,...,...
Adele,All I Ask,44,1061669,2015-11-2015,False
Adele,Set Fire to the Rain,105,1054230,2011-1-2011,False
Adele,Let It Go,258,758844,2013-11-2013,False
